# Objective
The goal of this personalization project will be to maximize the accuracy of our recommendation system when considering the results of recommending the top-5 movies for a given user using MovieLens' 10M ratings dataset.

Background: MovieLens is a web-based recommender system and virtual community that recommends movies for its users to watch, based on their film preferences using CF of members' movie ratings and reviews. To address the cold-start problem for new users, MovieLens uses preference elicitation where they ask new users to rate how much they enjoy watching different genres of movies.

The dataset contains 10000054 ratings. Users were selected at random but all users selected had rated at least 20 movies. The data contains three files: 

#### 1. Movies.dat
Each line of this file represents one movie, and has the following format: MovieID::Title::Genres
MovieID is the real MovieLens id.

Movie titles, by policy, should be entered identically to those found in IMDB, including year of release. However, they are entered manually, so errors and inconsistencies may exist.

Genres are a pipe-separated list, and are selected from the following:

Action
Adventure
Animation
Children's
Comedy
Crime
Documentary
Drama
Fantasy
Film-Noir
Horror
Musical
Mystery
Romance
Sci-Fi
Thriller
War
Western


#### 2. Ratings.dat
All ratings are contained in the file ratings.dat. Each line of this file represents one rating of one movie by one user, and has the following format:

UserID::MovieID::Rating::Timestamp

The lines within this file are ordered first by UserID, then, within user, by MovieID.

Ratings are made on a 1-5 star scale, with half-star increments.

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.


#### 3. Tags.dat
All tags are contained in the file tags.dat. Each line of this file represents one tag applied to one movie by one user, and has the following format:

UserID::MovieID::Tag::Timestamp

The lines within this file are ordered first by UserID, then, within user, by MovieID.

Tags are user generated metadata about movies. Each tag is typically a single word, or short phrase. The meaning, value and purpose of a particular tag is determined by each user.

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.



# 1. Item-Based Collaborative Filtering Algorithm

The general methodology we used to create an item-based collaborative filtering algorithm is as follows
1. Define a set of mutually observed users that have specified ratings for movie i and j 
2. Compute similarity between movie i and movie j using Pearson's correlation
3. Find top k matching items to movie i for which user u has specified ratings. The weighted average of these ratings is reported as the predicted rating value of that particular item.

Our objective is to maximize the accuracy of the top 5 movie recommendations users would enjoy out of those they have not seen yet. To do this we will use an approach that is similar to weighted KNN. 

In [8]:
import pandas as pd

moviescol = ['MovieId', 'Title', 'Genres','Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

ratings = pd.read_csv('./ratings.dat', sep = '::', names = ['UserId', 'MovieId', 'Rating', 'Timestamp'], engine = 'python')
movies = pd.read_csv('./movies.dat', sep ='::', names = moviescol, engine='python')

In order to determine the most appropriate way to sample our data, we first need to get a better understanding of the sparsity level of the user-item matrix. As seen in the introduction paper of item-based CF, we can calculate the Sparsity level of a matrix by using 1- (nonzero entries/total entries) 

In [9]:
# Compute the Sparsity of the Matrix by first finding the number of unique items and users present in the ratings df. Although the Movielens site already provides us with number of unique movies and usrs information, we provide a procedure to obtain it regardless.

n_users = ratings['UserId'].nunique()
n_items = ratings['MovieId'].nunique()

print 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)

sparsity = round(1.0-len(ratings)/float(n_users*n_items),3)
print 'The sparsity level of MovieLens10M is ' +  str(sparsity*100) + '%'

Number of users = 730 | Number of movies = 6373
The sparsity level of MovieLens10M is 97.9%


Next we'll load a python file in which we define all of our useful functionality. See `funcs.py` in the repository for the source. We use the surprise library for building and training the KNN model. 


In [ ]:
# %load 'funcs.py'
import surprise as sp
from surprise import AlgoBase
import pandas as pd
import numpy as np
from sklearn import cross_validation as cv
import funcs as F
import itertools as it

def build_movie_genre_matrix(movies):
    """
    Build a NxM matrix, rows are movie_ids, columns are genres, values 0, 1
    @param movies Dataframe dataframe of movie data
    @returns Matrix like Dataframe with 0s and 1s filled in 
    """
    movie_genre = []
    for (idx, row) in movies.iterrows(): 
        genres = row.loc['Genres'].split("|")
        movieid = row.loc['MovieId']
        for g in genres:  
            movie_genre.append({'MovieId': movieid, 'Genre': g})

    moviegenrecol = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
    test = pd.DataFrame(0, index = np.arange(len(movies)), columns = moviegenrecol)
    MovieGenres = pd.concat([movies['MovieId'], test], axis = 1)
    MovieGenres.columns= ['MovieId','Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

    for row in movie_genre: 
        movieID = row['MovieId']
        genre = row['Genre']
        MovieGenres.loc[MovieGenres.MovieId == movieID, genre] = 1

    return MovieGenres

        
def build_user_item_matrix(ratings):
    """
    Return a USERxITEM matrix with values as the user's value for the movie, null otherwise
    Right now not normalized
    @param ratings Dataframe
    @returns matrix numpy matrix with a user's ratings per movie
    """
    matrix = ratings.pivot(index = 'UserId', columns = 'MovieId', values = 'Rating').fillna(0)
    return matrix

def sample(ratings, user_counts, movie_counts, n, m):
    """
    Return a smaller matrix with top n users and top m items only
    @param ratings the ratings dataset 
    @param user_counts Count values of user ratings
    @param movie_counts Movie count values
    @param n number of users with most ratings
    @param m number of movies with most ratings
    @returns NxM matrix of USERxITEM ratings
    """
    n_users = ratings['UserId'].nunique()
    n_items = ratings['MovieId'].nunique()

    user_sample = user_counts.head(n).index
    movie_sample = movie_counts.head(m).index

    print len(user_sample)
    print len(movie_sample)

    subset = ratings.loc[ratings['UserId'].isin(user_sample)].loc[ratings['MovieId'].isin(movie_sample)]
    # we don't need the timestamp
    del subset['Timestamp']
    return subset


# define our own baseline algorithm that Surprise can use. Just predict the average!
class AvgBase(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

    # see Surprise#building_custom_algo
    def train(self, trainset):
        AlgoBase.train(self, trainset)
        # remember DF is uid, iid, rating
        self.mean = np.mean([rating for (_, _, rating) in self.trainset.all_ratings()])

    def estimate(self, u, i):
        return self.mean


def normalize_user_means(ratings):
    """
    @param Ratings pandas dataframe
    @returns user mean normalized dataframe
    """
    u_i_df = F.build_user_item_matrix(ratings)
    means = u_i_df.mean(axis=1)

    def f(row):
        u_id = row["UserId"]
        new_r = row["Rating"] - means[u_id]
        return new_r

    ratings['Rating'] = ratings.apply(f, axis = 1)
    return ratings
    

# we need something to test our model against. Use above defined model.
def train_baseline(ratings):
    """
    Baseline model. Same as below, return a model and data to test it on
    @param ratings Pandas Dataframe with UserId, MovieId, Ratings
    @returns Tuple (algorithm, testdata) basemodel that just returns the baseline estimate for user/item (adjusted for user bias)
    """
    train_data, test_data = cv.train_test_split(ratings, test_size = 0.20)

    algo = AvgBase()
    reader = sp.Reader(rating_scale=(1, 5))

    trainset = sp.Dataset.load_from_df(ratings, reader)
    testset = sp.Dataset.load_from_df(test_data, reader)

    trainset = trainset.build_full_trainset()

    algo.train(trainset)

    testset = testset.build_full_trainset().build_testset()
    return (algo, testset)



# train the KNN model on subsets of the data (for cross-validation)
def train(ratings, k_neighbors, k_folds):
    """
    Train a model and return it. Then we can use the model and evaluate it elsewhere
    @param ratings dataframe pandas dataframe to train on, with columns UserId, MovieId, Ratings
    @param k_neighbors number of neighbors to examine
    @param k_folds number of folds for cross validation
    @returns List of (algo, test data)
    We can call methods such as `test` and `evaluate` on this object 
    """

    train_data, test_data = cv.train_test_split(ratings, test_size = 0.20)
    reader = sp.Reader(rating_scale=(1, 5))

    trainset = sp.Dataset.load_from_df(train_data, reader)
    testset = sp.Dataset.load_from_df(test_data, reader)

    trainset.split(n_folds = k_folds)

    similarity_options = { 'name': 'pearson', 'user_based': False }
    algo = sp.KNNWithMeans(sim_options = similarity_options, k = k_neighbors, min_k = 5)

    for _trainset, _ in trainset.folds():
        algo.train(_trainset)


    testset = testset.build_full_trainset().build_testset()
    return (algo, testset)

def train_matrix(ratings, factor, k_folds):
    """
    Train a model and return it. Then we can use the model and evaluate it elsewhere
    @param ratings dataframe pandas dataframe to train on, with columns UserId, MovieId, Ratings
    @param n_folds number of folds for cross validation
    @returns List of (algo, test data)
    We can call methods such as `test` and `evaluate` on this object 
    """

    train_data, test_data = cv.train_test_split(ratings, test_size = 0.20)
    reader = sp.Reader(rating_scale=(1, 5))

    trainset = sp.Dataset.load_from_df(train_data, reader)
    testset = sp.Dataset.load_from_df(test_data, reader)
    trainset.split(n_folds = k_folds)

    algo = sp.SVD(n_factors = factor)

    for trainset, _ in trainset.folds():
        algo.train(trainset)
        
    testset = testset.build_full_trainset().build_testset()
    return (algo, testset)


def group_predictions_by_user(predictions):
    """
    @param List of Surprise predictions objects
    @returns Dict {uid: [P1, P2, ...PN]} hash mapping user id to top n predictions
    """
    p = sorted(predictions, key = lambda x: x.uid)

    groups = {}
    for k, g in it.groupby(p, lambda x: x.uid):
        groups[k] = sorted(list(g), key = lambda x: x.est, reverse = True)

    return groups


# For every item that a user would be rated, in top k
def calculate_catalog_coverage(ratings, predictions, k):
    """
    Calculate the catalog coverage of a model over a dataset
    @param ratings pandas dataframe with UserId, MovieId, Ratings. Must be the same set the model was trained on
    @param List Surprise predictions
    @oaram k Int the top k recommendations size
    @returns Float percentage of items recommended to at least one user
    """
    n_movies = ratings['MovieId'].nunique()

    movies_reccommended = set() # keep track of which movies are recommended. Note we only care about the number

    recommendations = group_predictions_by_user(predictions)
    for u_id, recs in recommendations.iteritems():
        movies_reccommended.update(map(lambda x: x.iid, recs[0:3]))

    return len(movies_reccommended) / float(n_movies)


# print some evaluative summaries
def evaluate(algo, ratings, testset, top_k = 5):
    """
    @param algo Surprise algorithm the model that was trained
    @oaram ratings The ratings it was trained on, in pandas Dataframe form (so we can calculate coverage)
    @param testset Surprise testset object, the data held out during cross-validation
    @returns Nested Dictionary {test: {rmse, mae}, train: {rmse, mae, cc}}
    We can use these to build up arrays for plotting.
    """

    ret = {}
    ret['test'] = {}
    ret['train'] = {}

    test_predictions = algo.test(testset)
    # see how it would do on the trainset to compare, comes with the algo object
    trainset = algo.trainset.build_testset()
    train_predictions = algo.test(trainset)

    # sticking evaluate in everything for grep, training is verbose
    ret['test']['rmse'] = sp.accuracy.rmse(test_predictions)
    ret['train']['rmse'] = sp.accuracy.rmse(train_predictions)

    ret['test']['mae'] = sp.accuracy.mae(test_predictions)
    ret['train']['mae'] = sp.accuracy.mae(train_predictions)

    # Hackish, baseline does not have a sense of "neighbors"
    knn = algo.__module__ = "surprise.prediction_algorithms.knns"
    mf =  algo.__module__ = "surprise.prediction_algorithms.matrix_factorization"

    if (knn or mf):
        ret['test']['cc'] = calculate_catalog_coverage(ratings, test_predictions, top_k)

    return ret




# matrix = funcs.build_user_item_matrix(ratings, movies)


Given that we want to work with a sample size of 10000 users and 100 items to start with, we cannot sample at random due to the high sparsity level previously calculated. Therefore we will obtain the top 10000 users with the most ratings and the most rated 100 movies. Funcs.py defines just such a sampling function, taking the top n users with the most ratings and the top-m most rated movies. In order to sample we need to know the top ratings N users and top rated M movies respectively. We can do this inside the sampling function, but since this doesn't change across sample sizes, we'll do it once outside the function and pass the value counts in.

In [11]:
user_counts = ratings['UserId'].value_counts()
movie_counts = ratings['MovieId'].value_counts()

subset = sample(ratings, user_counts, movie_counts, 10000, 100)

And then we can train our model using a function defined above. `train(data, k_neighbors, k_folds)` handles splitting the data into train and test sets, and folds for cross validation. This function trains our model, and then returns to us a tuple containing the (model, testdata) i.e. the model and the 20% of test data that we have withheld, so that way can test it and do what we like with the predictions. 

The library calculates mean centered pearson similarity under the hood. Conceptually it is simply mean centering all of the items, and computing the pairwise similarity for each, based on user ratings. This is the slowest part of the model, running in `O(nm^2)` time, where m is the number of items and n is the number of users. This is the reason that item-based methods are more tractable than user-based methods; the number of items is usually far less than the number of users. `nm^2` is still pretty big, but since this gets run offline that's alright, as long as it runs on the order of hours rather than weeks.

In [12]:
model, testdata = train(subset, 10, 5)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


Now we can evaluate our model. The function defined above `evaluate(algo, rating, testset, top_k=5)` takes the trained model, the original dataset, a testset, and the number of top-k recs and comptues some useful summaries about our model. Specifically it returns a hash with RMSE, MAE for test and train sets, and catalog coverage for the test set. 

In [13]:
evaluation = evaluate(model, subset, testdata)

RMSE: 0.8508
RMSE: 0.5245
MAE:  0.6501
MAE:  0.4068


In [14]:
print evaluation

{'test': {'cc': 0.99, 'mae': 0.65010674683631375, 'rmse': 0.85077590085280186}, 'train': {'mae': 0.40683865954621445, 'rmse': 0.52454923642969709}}


# 2. Model Based CF Algorithm
Matrix Factorization

# 3. Evaluation Methods

## (i) Cross Validation Setup
  The `surprise` library handles cross validation for us. After we split the ratings dataframe into train, and test (80%, 20%), `surprise` gives you the option to split the training data into different folds. We can iterate over these folds and train successively. This ensures that we're training over a balanced set. Finally, we withhold the test data from training and return it along with the model so that we can test it. 



## (ii) Accuracy on Training/Test Data
For accuracy we can take a look at MAE and RMSE of the models over a baseline. Our `evaluate` method returns a hash with the mae, rmse for the test and train predictions. We ran our models (offline) across a wide range of hyperparameters, using k-values and latent factors of [5, 10, 15...55]. Some of the results are plotted below. 

The errors are much smaller on training data for obvious reasons, but even on test data we beat the baseline by a large amount. MAEs were as low as 0.50 for some iterations of KNN and MF on the subset with 10,000 users and 2,000 items. For reference, the baseline, the simple average across all ratings, was around 0.80 for most runs.

The code used to generate the offline predictions appears below, taken from `script.py` in the repository. If one wants to run the model, simply run `python script.py` in the shell. This will take several hours if the larger subsets are included.   


The size of the neighborhood has significant impact on the prediction quality. So to determine the sensitivity of this parameter, we varied the number of neighbors used and observed its effect on MAE. Our results are shown below

In [ ]:
import matplotlib.pyplot as plt

#MAE vs Neighborhood size
plt.plot(k, Sample1MAE, k, Sample2MAE, k, Sample3MAE)
plt.title('Sensitivity of the Neighborhood Size')
plt.show()


In [2]:
import json
test = pd.read_json(/forKathyKnnVsErr.json, orient = 'index')

SyntaxError: invalid syntax (<ipython-input-2-56862f28282e>, line 2)

Similarity, the number of latent factors used for our matrix factorization impacts our prediction quality as well. We analyzed the sensitivity of this parameter

In [ ]:
#MAE vs Neighborhood size
plt.plot(f, Sample1MAE, f, Sample2MAE, f, Sample3MAE)
plt.title('Sensitivity of the Neighborhood Size')
plt.show()


## (iii) Coverage on training and test data
	a. User-Coverage: the fraction of users for which AT LEAST k items can be recommended well 
	b. Item-Coverage: the fraction of items that can be recommended to at least k users well
	c. Catalog Coverage: the fraction of items that are in the top-k for at least 1 user
    


How do your evaluation metrics change as a function of parameters such as neighborhood size, # of latent dimensions? 

_Personal Note_ In general when the neighborhood size K is small, we're forcing our classifier to be "more blind" to the overall distribution. A small K will have low bias but higher variance. On the other hand, a higher K averages more voters in each prediction and is more resilient to outliers. This consequently results in lower variance but increased bias. 

### Model Size Variation
How does overall accuracy change when you systematically sample your data from a small to large size? How does runtime scale? 

As mentioned earlier, intuitively we'd expect the runtime of both algorithms to increase as our model size increases. The figure below validates this intuitoin and compares the runtime of our Item Based model against its model based counterpart. 

In [ ]:
#MAE vs Neighborhood size
plt.plot(k, Sample1MAE, k, Sample2MAE, k, Sample3MAE)
plt.title('Sensitivity of the Neighborhood Size')
plt.show()